### IBL example for a basket of ETF's 

These setups are required to run this example in windows Python

In [1]:
import pandas as pd
import numpy as np

In [2]:
pip install git+https://github.com/snayeri/IBL.git#egg=IBL

  Cloning https://github.com/snayeri/IBL.git to c:\users\spk26\appdata\local\temp\pip-install-yw18hzaw\ibl
Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: six>=1.5 in c:\users\spk26\anaconda3\lib\site-packages (from python-dateutil>=2.6.1->pandas->IBL) (1.15.0)
  Created wheel for IBL: filename=IBL-0.1-py3-none-any.whl size=242755 sha256=d786ac036ded66e509c90ccd7086c980c90257892ae09324c46b4d0b8fba01a3
  Stored in directory: C:\Users\spk26\AppData\Local\Temp\pip-ephem-wheel-cache-ocvrfy6y\wheels\00\95\8b\a7db92f7ee31cbaf808f18ec3350a1f76b28162e1cc5414f60
Successfully built IBL



  Running command git clone -q https://github.com/snayeri/IBL.git 'C:\Users\spk26\AppData\Local\Temp\pip-install-yw18hzaw\IBL'


For this example, daily prices of the following securities for period 01-30-2009 to 04-01-2021 are loaded from the package. The data is downloaded from yahoo using pandas_datareader:

1. portfolio dataframe consists of the following ETFs: 

| ETF | representation |
| :--- | :-------------- |
| SPY | large cap US equities |
| VB | small cap US equities |
| FXI | large cap Chinese equities |
| VGK | Europe equities |
| EWJ | Japan equities |
| VWO | emerging market equities |
| LQD | US corporate bonds |
| IGOV | International treasury bonds |
| GLD | commodities |
|SLV | commodities |

2. market is represented by ACWI
3. marketcap is the total number of shares for each security multiplied by its last price
4. riskfree is derived from quotes for 90 day t-bills, converted to daily risk free rate, according to [convention](https://www.treasurydirect.gov/instit/annceresult/press/preanre/2004/ofcalc6decbill.pdf)

In [3]:
import IBL.data.portfolio
portfolio = IBL.data.portfolio.load()
import IBL.data.market
market = IBL.data.market.load()
import IBL.data.marketcap
marketcap = IBL.data.marketcap.load()
import IBL.data.riskfree
riskfree = IBL.data.riskfree.load()

Next step is to import the Idzorek_BL model and place the above data in the model

In [4]:
from IBL import IBL_model

In [5]:
example = IBL_model(portfolio.loc[:'2021-03-31'],marketcap,market.loc[:'2021-03-31'],riskfree,0.025)

### now let's add a view
let's assume we believe that over the next month, with 50% confidence, SPY will outperform IGOV by 1%
First thing is to define a dataframe with view vector(rho) and corresponding confidence  

In [6]:
outperformer = 'SPY'
underperformer = 'IGOV'
rhh = pd.DataFrame(index=portfolio.columns)
rhh.loc[:,'rho']=0
rhh.loc[:,'confidence'] =0
rhh.loc[outperformer,'rho'] = 1
rhh.loc[underperformer,'rho'] = -1
rhh.loc[outperformer,'confidence'] = 0.50
rhh.loc[underperformer,'confidence'] = 0.50
rhh

,rho,confidence
SPY,1,0.5
VB,0,0.0
FXI,0,0.0
VGK,0,0.0
EWJ,0,0.0
VWO,0,0.0
LQD,0,0.0
IGOV,-1,0.5
GLD,0,0.0
SLV,0,0.0


In [7]:
example.views_expected_r(np.array([0.01]),np.array([rhh['rho']]),np.array([rhh['confidence']]))

array([0.0093559 , 0.01224927, 0.00955845, 0.01108404, 0.00712169,
       0.01114654, 0.00172324, 0.00207014, 0.00192365, 0.00890946])

These new expected return premiums' vector can now be used in portfolio optimzation